In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wrds

In [2]:
# !pip install wrds

In [3]:
# read stata file to dataframe
ibes_details = pd.read_stata(r"G:\Resaerch_Data\ibes\ibes_actual_2000_2022\xz65ck0irfad7jdr.dta")

# read target firms file to dataframe
data = pd.read_excel(r"G:\Resaerch_Data\erwin_collaboration\data_for_leo.xlsx")

# read ibes_recommend_2000_2022.dta to dataframe
ibes_rcmd = pd.read_stata(r"G:\Resaerch_Data\ibes\ibes_recommend_2000_2022\ibes_rcmd_2000_2022.dta")

# read ibes_crsp_link.dta to dataframe
ibes_crsp = pd.read_stata(r"G:\Resaerch_Data\ibes\ibes_crsp_link\w0wsv1psjo6mktts.dta")

# read ibes_summary_2000_2022.dta to dataframe
ibes_summary = pd.read_stata(r"G:\Resaerch_Data\ibes\ibes_sumary_2000_2022\syozstvbh17yh9pk.dta")

# read ibes_surprise_2000_2022.dta to dataframe
ibes_surprise = pd.read_stata(r"G:\Resaerch_Data\ibes\ibes_surprise_2000_2022\mw7xvacrc2pw6az2.dta")

In [4]:
data

,obsnum,ibes_ticker,ibes_cname,SCORE,gvkey,permno,ticker,company_name,dummy_2000,dummy_2007,CUSIP_orig,CUSIP_new,permno_erwin,UPDATED_TICKER_new
0,1,HSM,HUSSMANN INTL INC,0.0,66293.0,85767.0,999,HUSSMANN INTERNATIONAL INC,1,0,44811010,NaN,85767.0,NaN
1,2,AAON,AAON INC,0.0,21542.0,76868.0,AAON,AAON INC,1,0,00036020,NaN,76868.0,NaN
2,3,AAIT,ADVANCED ANALOGIC TECHNOLOGIES I,0.0,163679.0,90854.0,AATI,ADVANCED ANALOGIC TECH,0,1,NaN,00752J108,NaN,AATI
3,4,ACAT,ARCTIC CAT INC,0.0,22461.0,76207.0,ACAT,ARCTIC CAT INC,0,1,NaN,39670104,NaN,ACAT
4,5,ACLR,ACCENT COLOR SCIENCES INC,0.0,64171.0,84288.0,ACLR,ACCENT COLOR SCIENCES,1,0,00430510,NaN,84288.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,288,WWY,WM WRIGLEY JR CO,1.0,11609.0,15472.0,WWY,WRIGLEY (WM) JR CO,0,1,NaN,982526105,NaN,WWY
288,289,XXIA,IXIA INC,0.0,140759.0,88750.0,XXIA,IXIA,0,1,NaN,45071R109,NaN,XXIA
289,290,TELM,DZS INC,0.0,140979.0,89925.0,ZHNE,ZHONE TECHNOLOGIES INC,0,1,NaN,98950P108,NaN,ZHNE
290,291,ZIGO,ZYGO CORP,0.0,11694.0,84161.0,ZIGO,ZYGO CORP,1,0,98985510,NaN,84161.0,NaN


In [5]:
# check if the ticker and sdate are unique in ibes_crsp
ibes_crsp.drop_duplicates(subset=['TICKER','sdate'], keep='first', inplace=True)
# remove the row that score is above 1
ibes_crsp = ibes_crsp[ibes_crsp['SCORE']<=1]
len(ibes_crsp)

27475

In [6]:
# check the number of unique values in a certain column of ibes_rcmd
print("There are %d Estimators in IBES database."%len(ibes_rcmd['EMASKCD'].unique().tolist()))
print("There are %d Estimators in IBES database."%len(ibes_rcmd['ESTIMID'].unique().tolist()))
# check how many financial analysts in IBES database
print("There are %d financial analysts in IBES database."%len(ibes_rcmd['ANALYST'].unique().tolist()))
# check how many stocks covers in IBES database
print("There are %d OFTIC covers in IBES database."%len(ibes_rcmd['OFTIC'].unique().tolist()))
print("There are %d ticker covers in IBES database."%len(ibes_rcmd['TICKER'].unique().tolist()))
# check how many stock covers by a fiancial analyst on average
print("There are %f stocks covers by a fiancial analyst on average."%(len(ibes_rcmd['TICKER'].unique().tolist())/len(ibes_rcmd['ANALYST'].unique().tolist())))

There are 1064 Estimators in IBES database.
There are 1118 Estimators in IBES database.
There are 15144 financial analysts in IBES database.
There are 15330 OFTIC covers in IBES database.
There are 13820 ticker covers in IBES database.
There are 0.912573 stocks covers by a fiancial analyst on average.


In [7]:
1. # merge ibes_rcmd with ibes_crsp with ticker
# creat year column on both side
ibes_rcmd['year'] = ibes_rcmd['ACTDATS'].dt.year
ibes_crsp['year'] = ibes_crsp['sdate'].dt.year
ibes_rcmd_crsp = pd.merge(ibes_rcmd, ibes_crsp, how='left', on=['TICKER', 'year'])

In [8]:
ibes_rcmd_crsp.head(6)
len(ibes_rcmd_crsp)

616543

In [9]:
# target data left join ibes_rcmd
ibes_rcmd_data = pd.merge(ibes_rcmd, data, how='left', left_on='TICKER', right_on='ibes_ticker')
ibes_rcmd_data.head(5)

,TICKER,CUSIP,CNAME,OFTIC,ACTDATS,ESTIMID,ANALYST,ERECCD,ETEXT,IRECCD,...,gvkey,permno,ticker,company_name,dummy_2000,dummy_2007,CUSIP_orig,CUSIP_new,permno_erwin,UPDATED_TICKER_new
0,0000,87482X10,TALMER BANCORP,TLMR,2014-03-10,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000,87482X10,TALMER BANCORP,TLMR,2014-03-11,JPMORGAN,ALEXOPOULOS S,,OVERWEIGHT,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000,87482X10,TALMER BANCORP,TLMR,2014-03-11,KEEFE,MCGRATTY C,2,OUTPERFORM,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000,87482X10,TALMER BANCORP,TLMR,2014-03-11,RAYMOND,LONG D,1,STRONG BUY,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000,87482X10,TALMER BANCORP,TLMR,2014-03-11,STERNE,GREEN P,2,NEUTRAL,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# # create a new column called CUSIP which combines CUSIP_orig and CUSIP_new
data['CUSIP'] = data.apply(lambda x: x['CUSIP_orig'] if pd.isnull(x['CUSIP_new']) else x['CUSIP_new'], axis=1)

In [27]:
ibes_rcmd_data1 = pd.merge(ibes_rcmd, data, how='left', left_on='CUSIP', right_on='CUSIP_new', suffixes=('', '_target'))
# check how many tickers from data got matched in IBES
print("There are %d firms got covered in IBES"%len(ibes_rcmd_data1['ibes_ticker'].unique().tolist()))

print("There are %d firms got covered in IBES"%(len(ibes_rcmd_data1['CUSIP_orig'].unique().tolist())+len(ibes_rcmd_data1['CUSIP_new'].unique().tolist())))

print("There are %d firms got covered in IBES"%len(ibes_rcmd_data1['gvkey'].unique().tolist()))

There are 1 firms got covered in IBES
There are 2 firms got covered in IBES
There are 1 firms got covered in IBES


In [28]:
# check how many tickers from data got matched in IBES
print("There are %d firms got covered in IBES"%len(ibes_rcmd_data['ibes_ticker'].unique().tolist()))

print("There are %d firms got covered in IBES"%(len(ibes_rcmd_data['CUSIP_orig'].unique().tolist())+len(ibes_rcmd_data1['CUSIP_new'].unique().tolist())))

print("There are %d firms got covered in IBES"%len(ibes_rcmd_data['gvkey'].unique().tolist()))

There are 238 firms got covered in IBES
There are 93 firms got covered in IBES
There are 224 firms got covered in IBES


In [45]:
# check the duplicates in ibes_rcmd_data[['ibes_ticker','gvkey']]
unique_ticker = ibes_rcmd_data[['ibes_ticker','gvkey']].drop_duplicates(['ibes_ticker','gvkey'])
# drop the ticker if gvkey is null
unique_ticker = unique_ticker[~unique_ticker['gvkey'].isnull()]
# check the duplicated gvkey in unique_ticker
# unique_ticker[unique_ticker.duplicated(['gvkey'], keep=False)]
unique_ticker

,ibes_ticker,gvkey
16091,AAIT,163679.0
16202,AAON,21542.0
20575,ACAT,22461.0
22163,ACLS,137602.0
23734,ACU,1104.0
...,...,...
601302,WST,11376.0
603295,WWY,11609.0
608750,XXIA,140759.0
613024,ZIGO,11694.0


In [12]:
###################
# Connect to WRDS #
###################
# conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


In [ ]:
# gvkey_list = unique_ticker['gvkey'].tolist()
# cmp = conn.raw_sql("""select a.gvkey, a.iid, a.datadate, a.tic, a.conm, a.exchg, a.at, a.fyr, a.fic, a.sic, a.atq, a.epsfxy""") 